# Initialize the device

Demonstrate APDS9960 proximity and gesture detector

MANDATORY
```
  Linux export BLINKA_MCP2221=1
  Powershell $env:BLINKA_MCP2221=1
```
or
```
  Linux export BLINKA_U2IF=1
  Powershell $env:BLINKA_U2IF=1
```

Jupyter notebook demonstrating the APDS9960 proximity sensor
https://docs.circuitpython.org/projects/apds9960/en/latest/

Prerequisites:
```
python3 -m pip install ipympl
python3 -m pip install adafruit-circuitpython-apds9960
python3 -m pip install adafruit-circuitpython-neopixel
```


In [ ]:
%env BLINKA_U2IF=1
#%env BLINKA_MCP2221=1
# The Adafruit example says the above env setting will work but I have always had to set it on the shell that launches Juptyer

import time

import board
import busio
import adafruit_apds9960.apds9960
import neopixel

help(board)
# generic PICO UI2F SCL and SCA don't align with the Stemma connector on Keebs
#i2c = board.I2C(board.SCL, board.SDA)
# use the default
i2c = board.I2C()
sensor = adafruit_apds9960.apds9960.APDS9960(i2c)
# Tested on a KB2040 which has a neopixel
# A few of the Adafruit 2040 boards have neopixels
pixels = neopixel.NeoPixel(board.NEOPIXEL, 1)


# Brightness based on proximity - move your hand closer and farther
Use the neopixels as indicator

In [ ]:
# warm up(?) don't remember why this is here
sensor.enable_proximity = True
run_count = 100
print("starting run [", run_count, "]... ")
pre_run = time.perf_counter()
for run_num in range(run_count):
    raw_proximity = sensor.proximity
    # need to curve the brightness
    # Cannot talk to neopixels inside an animation
    if raw_proximity > 180:
        pixels[0] = (raw_proximity//2,0,0)
    elif raw_proximity > 64:
        pixels[0] = (raw_proximity//3,raw_proximity//3,0)
    else:
        pixels[0] = (0,0,raw_proximity)
    time.sleep(0.050)

post_run = time.perf_counter()
print(
    "take " + str(run_count) + " readings " + str(post_run - pre_run) + " secs"
)

sensor.enable_proximity = False
pixels[0] = (0,0,0)


# Plot proximity - move your hand closer and farther
This will plot the distance on the proximity sensor.

In [ ]:
# %matplotlib inline
# %matplotlib notebook
%matplotlib ipympl

sensor.enable_proximity = True

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import datetime
import matplotlib.dates as mdates
from collections import deque

# How many sensor samples we want to store
HISTORY_SIZE = 25

# Pause re-sampling the sensor and drawing for INTERVAL seconds
INTERVAL = 0.01

# Deque for X-Axis (time)
x_vals = deque(maxlen=HISTORY_SIZE)

# Deque for Y-Axis (accelerometer readings)
proximity_values = deque(maxlen=HISTORY_SIZE)

# Create 1 subplot
fig, (proximity_graph) = plt.subplots(1,1)

# Automatically adjust subplot parameters for nicer padding between plots
plt.tight_layout()

pixels[0] = (28,28,28)

def animate(i):
    # Poll the proximity sensor
    # 0 == max distance
    # 255 == as close as possible
    proximity_raw = sensor.proximity
    # invert it so that that very close is a smaller value like a distance
    proximity = 255 - proximity_raw

    # Add the proximity value to the proximity array
    proximity_values.append(proximity)

    # Grab the datetime, auto-range based on length of proximity array
    x_vals = [datetime.datetime.now() + datetime.timedelta(seconds=i) for i in range(len(proximity_values))]

    # Clear all axis
    proximity_graph.cla()

    # Set grid titles
    proximity_graph.set_title('Proximity', fontsize=10)

    # Enable subplot grid lines
    proximity_graph.grid(True, linewidth=0.5, linestyle=':')

    # Rotate and align x-axis tick labels so they look better
    fig.autofmt_xdate()

    # Display the sub-plots
    proximity_graph.plot(x_vals, proximity_values, color='r')

    # Pause the plot for INTERVAL seconds
    plt.pause(INTERVAL)

# Update graph every 125ms
# Run for 100 frames because we said repeat=False
ani = FuncAnimation(fig, func=animate, frames=100, repeat=False, cache_frame_data=False)

# can't disable here because this will run before the animation ends
# sensor.enable_proximity = False


# Gesture 

This will print thegesture direction for a a little while

In [ ]:
sensor.enable_color = False
sensor.enable_proximity = True
sensor.enable_gesture = True

run_count = 1000
for run_num in range(run_count):
  gesture = sensor.gesture()
  if gesture == 1:
    print("up")
  if gesture == 2:
    print("down")
  if gesture == 3:
    print("left")
  if gesture == 4:
    print("right")
  time.sleep(0.015)

print("We're out of here")



# Lets try color 

Hold a colored object over the sensor.  I have NOT had good results with this device

In [ ]:
sensor.enable_proximity = False
sensor.enable_gesture = False
sensor.enable_color = True

# https://docs.circuitpython.org/projects/apds9960/en/latest/api.html
# the driver default is 1
# sensor.color_gain = 1
# driver default is 256
# sensor.color_integration_time = 256

run_count = 500
while True:
    while not sensor.color_data_ready:
        time.sleep(0.005)

    # red, green, blue, clear
    r, g, b, c = sensor.color_data
    print("r: {}, g: {}, b: {}, c: {}".format(r, g, b, c))